In [ ]:
# Menginstall tabulate untuk pembuatan tabel
pip install tabulate

In [ ]:
# Mengimport library yang akan digunakan
import networkx as nx
import numpy as np
import pandas as pd

# Input jumlah guru dan kelas
m = int(input("Masukkan jumlah guru (m): "))
n = int(input("Masukkan jumlah kelas (n): "))

# Menghasilkan matriks p[i,j] secara acak (jumlah sesi tiap guru untuk tiap kelas)
p = np.random.randint(0, 6, (m, n))  # Asumsi batas seorang guru ngajar suatu kelas maksimal 5 kali
print("Matriks sesi yang harus dijadwalkan:")
print(p)

# Membuat daftar sesi yang harus dijadwalkan (guru, kelas) sebanyak p[i,j]
sessions = []
for i in range(m):
    for j in range(n):
        for _ in range(p[i, j]):  # Tambahkan sebanyak jumlah sesi yang diperlukan
            sessions.append((f"X{i+1}", f"Y{j+1}"))

# Buat graf di mana tiap sesi adalah titik
G = nx.Graph()
for idx, (guru1, kelas1) in enumerate(sessions):
    G.add_node((guru1, kelas1, idx))
    for jdx, (guru2, kelas2) in enumerate(sessions):
        if idx < jdx:
            # Kondisi terpenuhi jika:
            # 1. Kelas yang sama diajar oleh guru berbeda
            # 2. Guru yang sama mengajar kelas yang berbeda dalam satu periode
            if kelas1 == kelas2 or guru1 == guru2:
                G.add_edge((guru1, kelas1, idx), (guru2, kelas2, jdx))

# Pewarnaan graf untuk mendapatkan periode optimal
coloring = nx.coloring.greedy_color(G, strategy="largest_first")
num_periods = max(coloring.values()) + 1  # Jumlah periode yang dibutuhkan
print(f"Jumlah periode yang dibutuhkan dalam seminggu: {num_periods}")
print(f"Jumlah periode yang dibutuhkan per hari: {int(np.ceil(num_periods/5))}")

# Nama hari
hari_list = ["Senin", "Selasa", "Rabu", "Kamis", "Jumat"]

# Menyusun jadwal dalam format dictionary
days = 5  # 5 hari kerja
periods_per_day = int(np.ceil(num_periods / days))  # Pembagian periode per hari
schedule = {
    f"Jadwal {hari_list[d]}": {
        f"Periode {p+1}": {f"X{i+1}": "-" for i in range(m)}
        for p in range(periods_per_day)
    }
    for d in range(days)
}

# Memasukkan hasil pewarnaan ke dalam jadwal
for (guru, kelas, _), period in coloring.items():
    day = period % days  # Pastikan semua sesi tersebar merata di 5 hari
    period_in_day = period // days
    if period_in_day < periods_per_day:
        schedule[f"Jadwal {hari_list[day]}"][f"Periode {period_in_day+1}"][guru] = kelas

# Menampilkan jadwal dalam bentuk tabel yang ditranspose
from tabulate import tabulate

for day, periods in schedule.items():
    df = pd.DataFrame.from_dict(periods, orient='index').T
    df.index.name = "Guru"
    print(f"\n{day}:")
    print(tabulate(df, headers='keys', tablefmt='grid'))  # Tampilan grid seperti tabel nyata


Masukkan jumlah guru (m): 5
Masukkan jumlah kelas (n): 10
Matriks sesi yang harus dijadwalkan:
[[0 5 0 0 3 4 5 0 0 2]
 [4 3 3 0 2 0 5 4 0 4]
 [2 0 2 1 5 0 0 2 3 0]
 [2 1 2 2 2 5 0 2 0 2]
 [1 2 5 0 0 4 1 1 2 1]]
Jumlah periode yang dibutuhkan dalam seminggu: 25
Jumlah periode yang dibutuhkan per hari: 5

Jadwal Senin:
+--------+-------------+-------------+-------------+-------------+-------------+
| Guru   | Periode 1   | Periode 2   | Periode 3   | Periode 4   | Periode 5   |
+========+=============+=============+=============+=============+=============+
| X1     | Y6          | Y6          | Y2          | Y7          | -           |
+--------+-------------+-------------+-------------+-------------+-------------+
| X2     | Y5          | Y2          | Y7          | Y10         | Y1          |
+--------+-------------+-------------+-------------+-------------+-------------+
| X3     | Y3          | Y3          | Y5          | -           | -           |
+--------+-------------+---------